## STA 141B Lecture 9

## Part 1 Numpy and Vectorization


### Vectorization

- Numpy is matrix algebra package in Python
- Vectorization means converting an algorithm into matrix operations to speed it up
 - Python is slow: it is Just-in-time compiled (JIT)
 - C is fast: it is compiled and variables have static type
 - Vectorization just means that we notice that an operation has a recognizable form and we use the precompiled code for that operation
- basic operation: dot product between $n \times n$ matrices $A,B$

$$ (AB)_{ij} = \sum_{k=1}^n A_{ik} B_{kj}$$

Pseudocode takes $O(n^3)$ time...

```
For i=1,...,n:
  For j=1,...,n:
    Init C[i,j] = 0
    For k=1,...,n:
      C[i,j] += A[i,k] B[k,j]
```

In Python nested for loops are slow, in C they are fast (compiled, static typed)

Conclusion: implement in C, import in Python

In [1]:
import numpy as np
data_folder = 'data/'
seeds = np.loadtxt(data_folder + 'seeds_dataset.txt',dtype=np.float64)

Column meanings for the seeds dataset:
1. area A, 
2. perimeter P, 
3. compactness $C = 4\pi A/P^2$, 
4. length of kernel, 
5. width of kernel, 
6. asymmetry coefficient 
7. length of kernel groove.

In [2]:
n,p = seeds.shape #Remember tuple unpacking!
print("type:\t{}\nndim:\t{}\nshape:\t{}\nsize:\t{}\ndtype:\t{}".format(
    type(seeds),seeds.ndim,seeds.shape,seeds.size,seeds.dtype))

type:	<class 'numpy.ndarray'>
ndim:	2
shape:	(210, 8)
size:	1680
dtype:	float64


In [3]:
## Transpose
print(seeds.T.shape)
print(seeds.T)

(8, 210)
[[15.26   14.88   14.29   ... 13.2    11.84   12.3   ]
 [14.84   14.57   14.09   ... 13.66   13.21   13.34  ]
 [ 0.871   0.8811  0.905  ...  0.8883  0.8521  0.8684]
 ...
 [ 2.221   1.018   2.699  ...  8.315   3.598   5.637 ]
 [ 5.22    4.956   4.825  ...  5.056   5.044   5.063 ]
 [ 1.      1.      1.     ...  3.      3.      3.    ]]


In [4]:
rowsum = seeds.T @ np.ones(n)  # sum the rows
colsum = seeds @ np.ones(p)   # sum the columns

In [5]:
rowsum = seeds.sum(axis=0)
colsum = seeds.sum(axis=1)

In [6]:
def mult_ones(seeds):
    output = [0.]*p
    for row in seeds: #arrays are iterable and return the rows
        for i in range(p):
            output[i] += row[i]
    return output

In [7]:
%time output = mult_ones(seeds)

CPU times: user 2.02 ms, sys: 1.9 ms, total: 3.92 ms
Wall time: 503 µs


In [8]:
%time output = seeds.T @ np.ones(n)

CPU times: user 213 µs, sys: 219 µs, total: 432 µs
Wall time: 56.3 µs


In [9]:
%time output = seeds.sum(axis=0)

CPU times: user 218 µs, sys: 242 µs, total: 460 µs
Wall time: 87.7 µs


### Matrix Operations

In [10]:
## Some elementwise operations
decseeds = seeds/10.0 
mulseeds = 15*seeds - 1.

In [11]:
## Elementwise functions
declog = np.log(decseeds)
decexp = np.exp(decseeds)
decrt  = np.sqrt(decseeds)

In [12]:
ratvec = seeds[:,4] / seeds[:,5]
ratvec = ratvec.reshape((n,1))
seeds_rat = np.concatenate((seeds,ratvec), axis=-1)
ratvec.shape, seeds.shape

((210, 1), (210, 8))

In [13]:
everyten = seeds[0:n:10,:]
seeds_ten = np.concatenate((seeds,everyten), axis=0)

In [14]:
print("sum: {}\nmean: {}\nmin: {}\nmax: {}".format(
    seeds.sum(), seeds.mean(), seeds.min(), seeds.max()))

sum: 10557.3759
mean: 6.284152321428571
min: 0.7651
max: 21.18


In [15]:
print("sum: {}\nmean: {}\nmin: {}\nmax: {}".format(
    seeds.sum(axis=0), seeds.mean(axis=0), 
    seeds.min(axis=0), seeds.max(axis=0)))

sum: [3117.98   3057.45    182.9097 1181.992   684.307   777.0422 1135.695
  420.    ]
mean: [14.84752381 14.55928571  0.87099857  5.62853333  3.25860476  3.70020095
  5.40807143  2.        ]
min: [10.59   12.41    0.8081  4.899   2.63    0.7651  4.519   1.    ]
max: [21.18   17.25    0.9183  6.675   4.033   8.456   6.55    3.    ]


## Array Broadcasting

### When two matrices of different shapes are compared/multiplied/etc.

### Rules...
- add 1s to shape (left or right) until match
- replicate right axis until match

In [16]:
ssum = seeds.sum(axis=0)
ssum.shape

(8,)

In [17]:
## compare an array and a vector (or just lower dimensional array)
## pad 1 to the first axis
ssum - ssum.reshape((1,8))

array([[0., 0., 0., 0., 0., 0., 0., 0.]])

In [18]:
## Center the variable
## Replicates the lower dim array
seeds_center = seeds - seeds.mean(axis=0)

In [19]:
seeds.shape, seeds.mean(axis=0).shape

((210, 8), (8,))

## Matrix slicing, Views, Copies

In [20]:
seeds_copy = seeds.copy() #this will be explained later
print(seeds_copy[10:12,4:6])

[[3.242 4.543]
 [3.201 1.717]]


In [21]:
A = seeds_copy
A[1,1] = -11
print(seeds_copy[1,1])

-11.0


In [22]:
## Fancy Indexing
## Boolean and list indexing
myrows = [2,17,97]
print(seeds_copy[myrows,0:2])

[[14.29 14.09]
 [15.69 14.75]
 [18.98 16.57]]


In [23]:
## Index by a list
## Require same length since it matches
myrows = [2,17,97]
mycols = [1,2,3]
print(seeds_copy[myrows,mycols])

[14.09    0.9058  6.449 ]


In [24]:
## Boolean indexing
rowslarge = seeds[:,0]>20.2
print(seeds[rowslarge,0:2])

[[20.71 17.23]
 [21.18 17.21]
 [20.88 17.05]
 [20.97 17.25]
 [20.24 16.91]]


In [25]:
## Assignment of a slice
A = seeds_copy[1:5,4:6]
seeds_copy[1:5,4:6] = 999
print(seeds_copy[1:5,4:6])

[[999. 999.]
 [999. 999.]
 [999. 999.]
 [999. 999.]]


In [26]:
## What happenned to A?
A

array([[999., 999.],
       [999., 999.],
       [999., 999.],
       [999., 999.]])

In [27]:
A[:,:] = A * 2.
print(seeds_copy[1:5,4:6])

[[1998. 1998.]
 [1998. 1998.]
 [1998. 1998.]
 [1998. 1998.]]


- simple slicing creates a "view" of the array, a pointer to a subset of the other array
- fancy indexing produces copies, so modifying it does not modify the original
- can use `A.copy()` to explicitely produce copies

## Sorting and bisection search

- many sorting algorithms (quicksort, bubblesort, etc.)
- they work on array datatypes by moving around the elements until sorted
- once sorted you can easily search for elements, and insert while maintaining sort

<img width=60% src="Binary_Search_Depiction.png">

In [28]:
help(np.sort)

Help on function sort in module numpy:

sort(a, axis=-1, kind=None, order=None)
    Return a sorted copy of an array.
    
    Parameters
    ----------
    a : array_like
        Array to be sorted.
    axis : int or None, optional
        Axis along which to sort. If None, the array is flattened before
        sorting. The default is -1, which sorts along the last axis.
    kind : {'quicksort', 'mergesort', 'heapsort', 'stable'}, optional
        Sorting algorithm. The default is 'quicksort'. Note that both 'stable'
        and 'mergesort' use timsort or radix sort under the covers and, in general,
        the actual implementation will vary with data type. The 'mergesort' option
        is retained for backwards compatibility.
    
        .. versionchanged:: 1.15.0.
           The 'stable' option was added.
    
    order : str or list of str, optional
        When `a` is an array with fields defined, this argument specifies
        which fields to compare first, second, etc.  A si

In [29]:
help(np.searchsorted)

Help on function searchsorted in module numpy:

searchsorted(a, v, side='left', sorter=None)
    Find indices where elements should be inserted to maintain order.
    
    Find the indices into a sorted array `a` such that, if the
    corresponding elements in `v` were inserted before the indices, the
    order of `a` would be preserved.
    
    Assuming that `a` is sorted:
    
    ======  ============================
    `side`  returned index `i` satisfies
    ======  ============================
    left    ``a[i-1] < v <= a[i]``
    right   ``a[i-1] <= v < a[i]``
    ======  ============================
    
    Parameters
    ----------
    a : 1-D array_like
        Input array. If `sorter` is None, then it must be sorted in
        ascending order, otherwise `sorter` must be an array of indices
        that sort it.
    v : array_like
        Values to insert into `a`.
    side : {'left', 'right'}, optional
        If 'left', the index of the first suitable location found is g

### Exercise

Consider a 2D array where all of the elements of a given row are larger than all the elements of the previous one, and each row is sorted.  For example, the following satisfies this constraint,

```
[
[1,4,7],
[9,11,27],
[29,300,3000]
]
```
1. Write a method that rearranges the terms in an array such that it is in this format.
2. Write a method that tests if a given element is in the array (as efficient as you can).